In [67]:
import pandas as pd
import requests
from tqdm.auto import tqdm
tqdm.pandas()

In [68]:
def next_monday(date):
    return pd.date_range(start=date, end=date + pd.offsets.Day(6), freq='W-MON')[0]

# Setting

In [69]:
test_date = pd.to_datetime('2021-05-17')

In [70]:
models_to_exclude = ['COVIDhub-ensemble', 'COVIDhub-trained_ensemble', 
                     'CU-nochange', 'CU-scenario_high', 'CU-scenario_low', 'CU-scenario_mid']

locations_to_exclude = ["60", "66", "69", "72", "74", "78"]

# DC,11,District of Columbia
# AS,60,American Samoa
# GU,66,Guam
# MP,69,Northern Mariana Islands
# PR,72,Puerto Rico
# UM,74,U.S. Minor Outlying Islands
# VI,78,Virgin Islands

In [71]:
dtype={'target': str, 'location': str, 'type': str, 'quantile': float, 'value': float}
parse_dates=['forecast_date', 'target_end_date']

# Load Files

In [72]:
url = "https://api.github.com/repos/reichlab/covid19-forecast-hub/git/trees/master?recursive=1"
r = requests.get(url)
res = r.json()

In [73]:
files = [file["path"] for file in res["tree"] if (file["path"].startswith('data-processed/') and file["path"].endswith('.csv'))]

In [74]:
files[-5:]

['data-processed/epiforecasts-ensemble1/2021-04-19-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-04-26-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-03-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-10-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-17-epiforecasts-ensemble1.csv']

In [75]:
df_files = pd.DataFrame({'filename':files})

df_files['model'] = df_files.filename.apply(lambda f: f.split('/')[1])

df_files['forecast_date'] = df_files.filename.apply(lambda f: f.split('/')[2][:10])
df_files.forecast_date = pd.to_datetime(df_files.forecast_date)

df_files['timezero'] = df_files.forecast_date.apply(next_monday)

df_files = df_files[~df_files.model.isin(models_to_exclude)]

In [76]:
df_files

,filename,model,forecast_date,timezero
0,data-processed/AIpert-pwllnod/2020-12-21-AIper...,AIpert-pwllnod,2020-12-21,2020-12-21
1,data-processed/AIpert-pwllnod/2020-12-28-AIper...,AIpert-pwllnod,2020-12-28,2020-12-28
2,data-processed/AIpert-pwllnod/2021-01-04-AIper...,AIpert-pwllnod,2021-01-04,2021-01-04
3,data-processed/AIpert-pwllnod/2021-01-11-AIper...,AIpert-pwllnod,2021-01-11,2021-01-11
4,data-processed/AIpert-pwllnod/2021-01-18-AIper...,AIpert-pwllnod,2021-01-18,2021-01-18
...,...,...,...,...
3429,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-19,2021-04-19
3430,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-26,2021-04-26
3431,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-03,2021-05-03
3432,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-10,2021-05-10


### Test Data

In [77]:
VALID_TARGETS = [f"{_} wk ahead inc death" for _ in range(1, 5)] + \
                [f"{_} wk ahead cum death" for _ in range(1, 5)] + \
                [f"{_} wk ahead inc case" for _ in range(1, 5)]

In [78]:
df_test_files = df_files[df_files.timezero == test_date]

In [79]:
df_test_files.head()

,filename,model,forecast_date,timezero
21,data-processed/AIpert-pwllnod/2021-05-17-AIper...,AIpert-pwllnod,2021-05-17,2021-05-17
66,data-processed/BPagano-RtDriven/2021-05-16-BPa...,BPagano-RtDriven,2021-05-16,2021-05-17
114,data-processed/CEID-Walk/2021-05-17-CEID-Walk.csv,CEID-Walk,2021-05-17,2021-05-17
158,data-processed/CMU-TimeSeries/2021-05-17-CMU-T...,CMU-TimeSeries,2021-05-17,2021-05-17
217,data-processed/COVIDhub-baseline/2021-05-17-CO...,COVIDhub-baseline,2021-05-17,2021-05-17


In [80]:
dfs_test = []

for _, row in tqdm(df_test_files.iterrows(), total=df_test_files.shape[0]):
    #print(row['filename'])
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs_test.append(df_temp)

  0%|          | 0/56 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df_test = pd.concat(dfs_test)

In [ ]:
df_test.shape

In [ ]:
df_test

In [ ]:
# only consider US + 50 states
df_test = df_test[df_test.location.str.len() == 2]
df_test = df_test[~df_test.location.isin(locations_to_exclude)]

In [ ]:
df_test.shape

In [ ]:
df_test.location.nunique()

In [ ]:
# ensure that for all targets each model provides forecasts for all locations
df_test = df_test[df_test.groupby(['target', 'model'])['location'].transform('nunique') == 52]

In [ ]:
df_test = df_test[df_test.type == 'quantile']

In [ ]:
df_test['no_quantiles'] = df_test.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df_test['no_quantiles'] = df_test.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [24]:
df_test = df_test[(df_test.no_quantiles == 23) | 
        (df_test.target.str.contains('inc case') & (df_test.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [25]:
df_test.shape

(262808, 8)

In [26]:
# dict of the models available for each target
available_models = dict(df_test.groupby(['target'])['model'].unique())

In [27]:
available_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'IHME-CurveFit', 'JHUAPL-Bucky', 'JHU_IDD-CovidSP', 'Karlen-pypm',
        'LANL-GrowthRate', 'LNQ-ens1', 'MIT_CritData-GBCF',
        'Microsoft-DeepSTIA', 'OliverWyman-Navigator',
        'RobertWalraven-ESG', 'SteveMcConnell-CovidComplete',
        'UA-EpiCovDA', 'UMass-MechBayes', 'USC-SI_kJalpha',
        'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'JHU_IDD-CovidSP', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'MUNI-ARIMA', 'Microsoft-DeepSTIA', 'RobertWalraven-ESG',
        'UChicagoCHATTOPADHYAY-UnIT', 'USC-SI_kJalpha', 'UVA-Ensemble'],
       dtype=object),
 '1 wk ahead in

## Training Data

d - 4 weeks - (horizon - 1) --> end: d - horizon

In [28]:
lower_bound = test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(4 - 1))

In [29]:
lower_bound

Timestamp('2021-03-29 00:00:00')

In [30]:
df_files = df_files[(df_files.timezero >= lower_bound) & (df_files.timezero < test_date)].copy()

In [31]:
# assigns to each horizon the corresponding training forecast dates for the test date
h_dict = {}
for h in range(1, 5):
    h_dict[h] = [test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(h - 1)), 
                 test_date - pd.Timedelta(weeks=h)]

In [32]:
h_dict

{1: [Timestamp('2021-04-19 00:00:00'), Timestamp('2021-05-10 00:00:00')],
 2: [Timestamp('2021-04-12 00:00:00'), Timestamp('2021-05-03 00:00:00')],
 3: [Timestamp('2021-04-05 00:00:00'), Timestamp('2021-04-26 00:00:00')],
 4: [Timestamp('2021-03-29 00:00:00'), Timestamp('2021-04-19 00:00:00')]}

In [33]:
def relevant_horizons(d):
    hs = []
    for h in range(1, 5):
        if((d >= h_dict[h][0]) & (d <= h_dict[h][1])):
            hs.append(h)
    return hs

In [34]:
df_files['horizons'] = df_files.timezero.apply(relevant_horizons)

In [35]:
df_files

,filename,model,forecast_date,timezero,horizons
14,data-processed/AIpert-pwllnod/2021-03-29-AIper...,AIpert-pwllnod,2021-03-29,2021-03-29,[4]
15,data-processed/AIpert-pwllnod/2021-04-05-AIper...,AIpert-pwllnod,2021-04-05,2021-04-05,"[3, 4]"
16,data-processed/AIpert-pwllnod/2021-04-12-AIper...,AIpert-pwllnod,2021-04-12,2021-04-12,"[2, 3, 4]"
17,data-processed/AIpert-pwllnod/2021-04-19-AIper...,AIpert-pwllnod,2021-04-19,2021-04-19,"[1, 2, 3, 4]"
18,data-processed/AIpert-pwllnod/2021-04-26-AIper...,AIpert-pwllnod,2021-04-26,2021-04-26,"[1, 2, 3]"
...,...,...,...,...,...
3427,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-12,2021-04-12,"[2, 3, 4]"
3428,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-19,2021-04-19,"[1, 2, 3, 4]"
3429,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-26,2021-04-26,"[1, 2, 3]"
3430,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-03,2021-05-03,"[1, 2]"


In [36]:
# only keep relevant training data
df_files = df_files[df_files.horizons.apply(len) > 0]

In [37]:
dfs = []
for _, row in tqdm(df_files.iterrows(), total=df_files.shape[0]):
    VALID_TARGETS = [f"{_} wk ahead inc death" for _ in row['horizons']] + \
                    [f"{_} wk ahead cum death" for _ in row['horizons']] + \
                    [f"{_} wk ahead inc case" for _ in row['horizons']]
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs.append(df_temp)

  0%|          | 0/420 [00:00<?, ?it/s]

In [38]:
df = pd.concat(dfs)

In [39]:
df.shape

(9444823, 8)

# Data Cleaning

In [40]:
df = df[df.location.str.len() == 2]
df = df[~df.location.isin(locations_to_exclude)]

In [41]:
df.location.nunique() # US + 50 states + district of Columbia

52

In [42]:
df = df[df.type == 'quantile']

In [43]:
# how many forecasts for each target/model/location? should be 4 for every location
df['no_forecasts'] = df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique')

In [44]:
df = df[df.groupby(['target', 'model'])['no_forecasts'].transform('min') == 4].drop(columns='no_forecasts').reset_index(drop=True)

In [45]:
# df = df[df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique') == 4]
# would only remove the respective location with <4 forecasts, but we want to remove the model as a whole if in one location <4

In [46]:
df = df[df.groupby(['target', 'model', 'target_end_date'])['location'].transform('nunique') == 52]

In [47]:
df['no_quantiles'] = df.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df['no_quantiles'] = df.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [48]:
df[(df.target.str.contains('inc case') & (df.no_quantiles == 7))]

,forecast_date,target,target_end_date,quantile,type,value,location,model,no_quantiles
9048,2021-03-28,4 wk ahead inc case,2021-04-24,0.025,quantile,301688.453910,US,BPagano-RtDriven,7
9049,2021-03-28,4 wk ahead inc case,2021-04-24,0.100,quantile,373783.603670,US,BPagano-RtDriven,7
9050,2021-03-28,4 wk ahead inc case,2021-04-24,0.250,quantile,439303.742640,US,BPagano-RtDriven,7
9051,2021-03-28,4 wk ahead inc case,2021-04-24,0.500,quantile,512342.170200,US,BPagano-RtDriven,7
9052,2021-03-28,4 wk ahead inc case,2021-04-24,0.750,quantile,585380.597760,US,BPagano-RtDriven,7
...,...,...,...,...,...,...,...,...,...
1056091,2021-05-10,1 wk ahead inc case,2021-05-15,0.250,quantile,144960.921195,US,UVA-Ensemble,7
1056092,2021-05-10,1 wk ahead inc case,2021-05-15,0.500,quantile,234295.048101,US,UVA-Ensemble,7
1056093,2021-05-10,1 wk ahead inc case,2021-05-15,0.750,quantile,323629.175006,US,UVA-Ensemble,7
1056094,2021-05-10,1 wk ahead inc case,2021-05-15,0.900,quantile,404032.675928,US,UVA-Ensemble,7


In [49]:
df = df[(df.no_quantiles == 23) | 
        (df.target.str.contains('inc case') & (df.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [50]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    20
1 wk ahead inc case     16
1 wk ahead inc death    21
2 wk ahead cum death    20
2 wk ahead inc case     15
2 wk ahead inc death    21
3 wk ahead cum death    19
3 wk ahead inc case     16
3 wk ahead inc death    21
4 wk ahead cum death    19
4 wk ahead inc case     16
4 wk ahead inc death    21
Name: model, dtype: int64

In [51]:
# check if there are models used for training that are not available for the test date
train_models = dict(df.groupby('target').model.unique())

a = [(k, t) for k, v in train_models.items() for t in v]
b = [(k, t) for k, v in available_models.items() for t in v]

[i for i in a if i not in b]

[('1 wk ahead cum death', 'JHU_CSSE-DECOM'),
 ('2 wk ahead cum death', 'JHU_CSSE-DECOM'),
 ('3 wk ahead inc death', 'Columbia_UNC-SurvCon'),
 ('4 wk ahead inc death', 'Columbia_UNC-SurvCon')]

In [52]:
df = df[df.apply(lambda x: x.model in (available_models[x.target]), axis=1)]

In [53]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    19
1 wk ahead inc case     16
1 wk ahead inc death    21
2 wk ahead cum death    19
2 wk ahead inc case     15
2 wk ahead inc death    21
3 wk ahead cum death    19
3 wk ahead inc case     16
3 wk ahead inc death    20
4 wk ahead cum death    19
4 wk ahead inc case     16
4 wk ahead inc death    20
Name: model, dtype: int64

In [54]:
df.shape

(888940, 8)

In [55]:
models_by_target = pd.DataFrame([(i, k) for i,j in train_models.items() for k in j], 
                  columns=['target','model'])

In [56]:
models_by_target[models_by_target.target == '4 wk ahead cum death']

,target,model
169,4 wk ahead cum death,BPagano-RtDriven
170,4 wk ahead cum death,CEID-Walk
171,4 wk ahead cum death,COVIDhub-baseline
172,4 wk ahead cum death,CU-select
173,4 wk ahead cum death,Covid19Sim-Simulator
174,4 wk ahead cum death,CovidAnalytics-DELPHI
175,4 wk ahead cum death,DDS-NBDS
176,4 wk ahead cum death,JHUAPL-Bucky
177,4 wk ahead cum death,Karlen-pypm
178,4 wk ahead cum death,LANL-GrowthRate


In [57]:
df = df[~df.model.isin(models_to_exclude)]

In [58]:
df

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-03-28,4 wk ahead inc death,2021-04-24,0.010,quantile,4469.02672,US,BPagano-RtDriven
1,2021-03-28,4 wk ahead inc death,2021-04-24,0.025,quantile,5096.59716,US,BPagano-RtDriven
2,2021-03-28,4 wk ahead inc death,2021-04-24,0.050,quantile,5657.61206,US,BPagano-RtDriven
3,2021-03-28,4 wk ahead inc death,2021-04-24,0.100,quantile,6314.54213,US,BPagano-RtDriven
4,2021-03-28,4 wk ahead inc death,2021-04-24,0.150,quantile,6760.65562,US,BPagano-RtDriven
...,...,...,...,...,...,...,...,...
908071,2021-05-10,1 wk ahead cum death,2021-05-15,0.850,quantile,587081.00000,US,epiforecasts-ensemble1
908072,2021-05-10,1 wk ahead cum death,2021-05-15,0.900,quantile,587210.00000,US,epiforecasts-ensemble1
908073,2021-05-10,1 wk ahead cum death,2021-05-15,0.950,quantile,587436.00000,US,epiforecasts-ensemble1
908074,2021-05-10,1 wk ahead cum death,2021-05-15,0.975,quantile,587526.00000,US,epiforecasts-ensemble1


In [59]:
available_train_models = dict(df.groupby(['target'])['model'].unique())

In [60]:
available_train_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'JHUAPL-Bucky', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'MIT_CritData-GBCF', 'OliverWyman-Navigator', 'RobertWalraven-ESG',
        'SteveMcConnell-CovidComplete', 'UA-EpiCovDA', 'UMass-MechBayes',
        'USC-SI_kJalpha', 'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1', 'RobertWalraven-ESG',
        'UChicagoCHATTOPADHYAY-UnIT', 'USC-SI_kJalpha', 'UVA-Ensemble'],
       dtype=object),
 '1 wk ahead inc death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPH

In [61]:
df_test.shape

(262808, 8)

In [62]:
df_test = df_test[df_test.apply(lambda x: x.model in (available_train_models[x.target]), axis=1)]

In [63]:
df_test.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-05-16,1 wk ahead inc death,2021-05-22,0.010,quantile,2303.97704,US,BPagano-RtDriven
1,2021-05-16,1 wk ahead inc death,2021-05-22,0.025,quantile,2533.68444,US,BPagano-RtDriven
2,2021-05-16,1 wk ahead inc death,2021-05-22,0.050,quantile,2739.03074,US,BPagano-RtDriven
3,2021-05-16,1 wk ahead inc death,2021-05-22,0.100,quantile,2979.48454,US,BPagano-RtDriven
4,2021-05-16,1 wk ahead inc death,2021-05-22,0.150,quantile,3142.77389,US,BPagano-RtDriven


In [64]:
df.groupby(['target'])['model'].unique()

target
1 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
Name: model, dtype: object

# Export

In [65]:
df.to_csv('data/df_train_{}.csv'.format(str(test_date.date())), index=False)

In [66]:
df_test.to_csv('data/df_test_{}.csv'.format(str(test_date.date())), index=False)